# Jan-Sunwai AI — Dataset Re-Sorter v2 (Kaggle Edition)

Re-sorts the civic complaint image dataset using the **improved two-step Ollama AI pipeline** with all fixes:

### What's new in v2
| Fix | Description |
|-----|-------------|
| **New models** | `llama3.2-vision:11b` (vision) + `mistral:7b` (reasoning) — stronger than qwen2.5vl + llama3.1 |
| `format="json"` | Forces structured JSON output from Ollama — eliminates string matching failures |
| Hierarchical prompts | Groups → Departments — makes classification easier for the model |
| Keyword fallback | Scans vision description for civic keywords when reasoning returns Uncategorized |
| Real confidence | Uses model-reported confidence (0.0–1.0) instead of hardcoded values |
| Full audit trail | Logs method used, rationale, raw JSON for every image — full explainability |

### Models (Kaggle GPU — 16 GB VRAM)
| Step | Model | Size | Role |
|------|-------|------|------|
| Vision | `llama3.2-vision:11b` | ~6.7 GB | Meta's latest multimodal — superior visual understanding |
| Reasoning | `mistral:7b` | ~4.1 GB | Excellent at structured JSON output & instruction following |

Both models run sequentially (~10.8 GB peak VRAM), fits within 16 GB.

### Fast mode
Single-pass with `llama3.2-vision:11b` + `format="json"` — faster but slightly less accurate.

## Before running
1. The dataset has been split into **5 parts** (`backend/sorted_dataset_parts/part1` … `part5`).  
   Each part contains **all departments** with a random ~⅕ of each department's images (~1 GB per part).
2. Zip and upload each part as a separate Kaggle Dataset:
   - Kaggle → Datasets → New Dataset → upload `part1.zip` → name it e.g. `jan-sunwai-part1`
   - Repeat for `part2` … `part5`
3. Add all 5 datasets to this notebook: **Add Data → Your Datasets → select each one**
4. Update `PART_ROOTS` in Section 2 to match your dataset slugs if different
5. Enable **GPU accelerator**: **GPU P100** (recommended) or **GPU T4 x1**
6. Enable **Internet access** (Settings → Internet → On)
7. **Restart kernel** before running to avoid stale variables
8. Run All

## Output
- `/kaggle/working/ai_resort_report_v2.csv` — full classification audit trail
- `/kaggle/working/ai_sorted_dataset/<Category>/` — correctly sorted images

## 1 — Install Ollama and start server

In [ ]:
import subprocess, time, os

# Install pciutils (lspci) + zstd — both required before Ollama install
print('Installing pciutils + zstd...')
subprocess.run(['apt-get', 'install', '-y', 'pciutils', 'zstd'],
               check=True, capture_output=True)
print('Done.')

# Verify GPU is visible
print('\n--- GPU check (lspci) ---')
r = subprocess.run(['lspci'], capture_output=True, text=True)
gpu_lines = [l for l in r.stdout.splitlines() if 'NVIDIA' in l or 'VGA' in l or '3D' in l]
for l in gpu_lines:
    print(' ', l)
if not gpu_lines:
    print('  ⚠ No GPU found via lspci — check accelerator settings')

# Install Ollama
print('\nInstalling Ollama...')
result = subprocess.run('curl -fsSL https://ollama.com/install.sh | sh',
                        shell=True, capture_output=True, text=True)
print(result.stdout[-600:] if result.stdout else 'done')
if result.returncode != 0:
    print('STDERR:', result.stderr[-300:])

In [ ]:
# Confirm CUDA GPU is available
print('--- nvidia-smi ---')
r = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,driver_version',
                    '--format=csv,noheader'], capture_output=True, text=True)
if r.returncode == 0:
    print(r.stdout.strip())
else:
    print('nvidia-smi not found — models will run on CPU (much slower).')

# Start Ollama server with GPU env vars
gpu_env = {
    **os.environ,
    'OLLAMA_NUM_GPU': '999',
    'CUDA_VISIBLE_DEVICES': '0',
}
server = subprocess.Popen(['ollama', 'serve'],
                          stdout=subprocess.DEVNULL,
                          stderr=subprocess.DEVNULL,
                          env=gpu_env)
print(f'\nOllama server PID: {server.pid}')
time.sleep(5)

r = subprocess.run(['ollama', 'list'], capture_output=True, text=True)
print(r.stdout or 'Server ready')

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# MODE SELECTOR — choose 'fast' or 'accuracy'
# ═══════════════════════════════════════════════════════════════════
#   fast:     single-pass llama3.2-vision:11b with format='json'
#             ~3-4 s/img, good accuracy with JSON enforcement
#   accuracy: two-step llama3.2-vision:11b → mistral:7b with format='json'
#             ~6-8 s/img, best accuracy with hierarchical classification
# ═══════════════════════════════════════════════════════════════════

MODE = 'accuracy'   # <--- CHANGE THIS

if MODE == 'fast':
    VISION_MODEL    = 'llama3.2-vision:11b'   # single-pass with Meta's latest
    REASONING_MODEL = None
    SINGLE_PASS     = True
    EST_SECONDS     = 3.5
else:
    VISION_MODEL    = 'llama3.2-vision:11b'   # Meta's latest multimodal
    REASONING_MODEL = 'mistral:7b'             # excellent at structured JSON
    SINGLE_PASS     = False
    EST_SECONDS     = 7.0

print(f'MODE       = {MODE}')
print(f'Vision     = {VISION_MODEL}')
print(f'Reasoning  = {REASONING_MODEL or "(single-pass)"}')

print(f'\nPulling {VISION_MODEL}...')
subprocess.run(['ollama', 'pull', VISION_MODEL], check=True)

if REASONING_MODEL:
    print(f'\nPulling {REASONING_MODEL}...')
    subprocess.run(['ollama', 'pull', REASONING_MODEL], check=True)

print('\nLoaded models:')
subprocess.run(['ollama', 'list'])

# Quick GPU residency check
print('\n--- GPU VRAM usage ---')
r = subprocess.run(['nvidia-smi', '--query-compute-apps=pid,used_gpu_memory',
                    '--format=csv,noheader'], capture_output=True, text=True)
print(r.stdout.strip() if r.returncode == 0 else '(cannot query GPU processes)')

## 2 — Load dataset (multi-part Kaggle input)

In [ ]:
from pathlib import Path

IMAGE_EXTS = {'.jpg', '.jpeg', '.png', '.webp', '.bmp', '.tiff'}

# ── Multi-part dataset support ──────────────────────────────────
# Update slugs to match your actual Kaggle dataset names
PART_ROOTS = [Path(f'/kaggle/input/jan-sunwai-part{i}') for i in range(1, 6)]

INPUT_ROOTS = [p for p in PART_ROOTS if p.exists()]

if not INPUT_ROOTS:
    print('⚠  None of the expected dataset paths found. Available inputs:')
    for p in sorted(Path('/kaggle/input').iterdir()):
        print(' ', p)
    raise FileNotFoundError('No dataset parts found. Attach them and update PART_ROOTS.')

print(f'Found {len(INPUT_ROOTS)} / {len(PART_ROOTS)} dataset parts.')

OUTPUT_ROOT = Path('/kaggle/working/ai_sorted_dataset')
REPORT_CSV  = Path('/kaggle/working/ai_resort_report_v2.csv')
AUDIT_JSON  = Path('/kaggle/working/ai_audit_trail.json')

print(f'Output : {OUTPUT_ROOT}')
print(f'Report : {REPORT_CSV}')

# Count images per category across all parts
cat_counts: dict[str, int] = {}
total = 0
for root in INPUT_ROOTS:
    for folder in sorted(root.iterdir()):
        if folder.is_dir():
            n = len([p for p in folder.iterdir() if p.suffix.lower() in IMAGE_EXTS])
            cat_counts[folder.name] = cat_counts.get(folder.name, 0) + n
            total += n

for cat, n in sorted(cat_counts.items()):
    print(f'  {cat:<45} {n:>5} images')

print(f'\nTotal : {total} images across {len(INPUT_ROOTS)} parts')
print(f'Est.  : ~{total * EST_SECONDS / 3600:.1f} h at ~{EST_SECONDS:.1f} s/img')

## 3 — Classifier (self-contained, all fixes included)

In [ ]:
# Install Python dependencies
subprocess.run(['pip', 'install', '-q', '--upgrade', 'ollama', 'pillow'], check=True)

In [ ]:
import io
import json
import ollama
from PIL import Image

# ══════════════════════════════════════════════════════════════════
# CANONICAL CATEGORIES (11 civic complaint departments)
# ══════════════════════════════════════════════════════════════════

CANONICAL_CATEGORIES = [
    'Municipal - PWD (Roads)',
    'Municipal - Sanitation',
    'Municipal - Horticulture',
    'Municipal - Street Lighting',
    'Municipal - Water & Sewerage',
    'Utility - Power (DISCOM)',
    'State Transport',
    'Pollution Control Board',
    'Police - Local Law Enforcement',
    'Police - Traffic',
    'Uncategorized',
]

CATEGORY_DEFINITIONS = {
    'Municipal - PWD (Roads)':        'broken roads, potholes, cracked pavement, damaged footpaths, bridge damage',
    'Municipal - Sanitation':         'garbage dumps, overflowing trash bins, dirty public toilets, waste piles on streets',
    'Municipal - Horticulture':       'fallen or uprooted trees, overgrown vegetation, unmaintained parks, dead/dry plants',
    'Municipal - Street Lighting':    'broken street lights, non-functional lamp posts, dark or unlit public roads',
    'Municipal - Water & Sewerage':   'waterlogging, flooded streets, blocked drains, sewer overflow, water pipe leaks',
    'Utility - Power (DISCOM)':       'dangling electrical wires, open or damaged transformers, hazardous power cables',
    'State Transport':                'damaged bus shelters, broken state buses, transport terminal damage',
    'Pollution Control Board':        'air pollution, thick smoke, industrial waste dumping, open burning of garbage',
    'Police - Local Law Enforcement': 'illegal parking, footpath encroachment, public nuisance, fights or brawls',
    'Police - Traffic':               'failed traffic signals, road blockages, severe traffic congestion',
    'Uncategorized':                  'does not clearly match any of the above civic categories',
}

# ══════════════════════════════════════════════════════════════════
# ALIAS MAP — normalises model output to canonical names
# ══════════════════════════════════════════════════════════════════

_ALIAS_MAP = {
    'municipal - pwd (roads)':         'Municipal - PWD (Roads)',
    'municipal - pwd roads':           'Municipal - PWD (Roads)',
    'municipal - sanitation':          'Municipal - Sanitation',
    'municipal - horticulture':        'Municipal - Horticulture',
    'municipal - street lighting':     'Municipal - Street Lighting',
    'municipal - water & sewerage':    'Municipal - Water & Sewerage',
    'municipal - water and sewerage':  'Municipal - Water & Sewerage',
    'utility - power (discom)':        'Utility - Power (DISCOM)',
    'utility - power discom':          'Utility - Power (DISCOM)',
    'state transport':                 'State Transport',
    'pollution control board':         'Pollution Control Board',
    'police - local law enforcement':  'Police - Local Law Enforcement',
    'police - traffic':                'Police - Traffic',
    'uncategorized':                   'Uncategorized',
}

def canonicalize(label: str) -> str:
    """Map model output to canonical category. Handles partial / fuzzy matches."""
    cleaned = label.strip().lower()
    if cleaned in _ALIAS_MAP:
        return _ALIAS_MAP[cleaned]
    # Fuzzy: check if any alias is a substring or vice versa
    for alias, canonical in _ALIAS_MAP.items():
        if cleaned in alias or alias in cleaned:
            return canonical
    return 'Uncategorized'


# ══════════════════════════════════════════════════════════════════
# FIX 1: KEYWORD FALLBACK — catches what reasoning model misses
# ══════════════════════════════════════════════════════════════════

_KEYWORD_FALLBACK = [
    (['pothole', 'road damage', 'cracked road', 'broken road', 'damaged road',
      'damaged pavement', 'broken pavement', 'footpath damage', 'manhole'],
     'Municipal - PWD (Roads)'),
    (['waterlog', 'flooded', 'flood', 'drain overflow', 'sewer overflow',
      'pipe leak', 'water gushing', 'stagnant water', 'blocked drain',
      'drainage problem'],
     'Municipal - Water & Sewerage'),
    (['garbage', 'trash', 'waste', 'litter', 'dump', 'rubbish', 'overflowing bin'],
     'Municipal - Sanitation'),
    (['fallen tree', 'uprooted tree', 'overgrown', 'dead plant', 'broken branch',
      'tree blocking'],
     'Municipal - Horticulture'),
    (['street light', 'lamp post', 'unlit road', 'broken light', 'dark road'],
     'Municipal - Street Lighting'),
    (['dangling wire', 'hanging wire', 'open transformer', 'fallen electric pole',
      'exposed wire', 'power cable'],
     'Utility - Power (DISCOM)'),
    (['smoke', 'burning', 'industrial waste', 'air pollution', 'open burning'],
     'Pollution Control Board'),
    (['traffic signal', 'signal failure', 'traffic jam', 'road blockage'],
     'Police - Traffic'),
    (['illegal parking', 'encroachment', 'footpath blocked', 'public nuisance'],
     'Police - Local Law Enforcement'),
    (['bus shelter', 'state bus', 'transport terminal'],
     'State Transport'),
]

_NEGATIVE_KEYWORDS = [
    'selfie', 'portrait', 'food', 'meal', 'restaurant', 'cartoon', 'anime',
    'gaming', 'screenshot', 'indoor furniture', 'appliance', 'pet', 'animal',
    'beautiful landscape', 'clear sky',
]

def keyword_fallback(description: str) -> str:
    """Scan vision description for civic keywords → best matching category."""
    desc = description.lower()
    for keywords, category in _KEYWORD_FALLBACK:
        if any(kw in desc for kw in keywords):
            return category
    return 'Uncategorized'


# ══════════════════════════════════════════════════════════════════
# HELPER FUNCTIONS
# ══════════════════════════════════════════════════════════════════

def safe_dirname(label: str) -> str:
    return label.replace(' ', '_').replace('(', '').replace(')', '').replace('&', 'and')

def load_as_jpeg_bytes(path: Path) -> bytes:
    with Image.open(path) as img:
        if img.mode != 'RGB':
            img = img.convert('RGB')
        buf = io.BytesIO()
        img.save(buf, format='JPEG', quality=85)
        return buf.getvalue()


# ══════════════════════════════════════════════════════════════════
# FIX 2: HIERARCHICAL CLASSIFICATION PROMPT (for reasoning step)
# ══════════════════════════════════════════════════════════════════

def _build_reasoning_prompt(description: str) -> str:
    """Hierarchical prompt: first identify group, then specific department."""
    categories_block = '\n'.join(
        f'- {cat}: {CATEGORY_DEFINITIONS[cat]}' for cat in CANONICAL_CATEGORIES
    )
    return (
        f'You are a civic complaint classifier for Indian municipal authorities.\n\n'
        f'Image description: "{description}"\n\n'
        f'First determine the department GROUP, then the specific department.\n\n'
        f'Groups:\n'
        f'- Municipal: city infrastructure (roads, sanitation, horticulture, lighting, water/sewerage)\n'
        f'- Police: law enforcement and traffic management\n'
        f'- Utility: electrical power distribution (wires, transformers)\n'
        f'- State Transport: public bus/transport infrastructure\n'
        f'- Pollution Control Board: air/water/soil pollution\n'
        f'- Uncategorized: no civic issue visible or unrecognisable image\n\n'
        f'Specific categories:\n'
        f'{categories_block}\n\n'
        f'Decision rules:\n'
        f'1. Dangling wires, power cables, transformer, electric pole → Utility - Power (DISCOM)\n'
        f'2. Waterlogging, flood, drain overflow, sewer, pipe leak → Municipal - Water & Sewerage\n'
        f'3. Garbage, trash, waste, dump, litter, bins → Municipal - Sanitation\n'
        f'4. Potholes, road cracks, broken road, pavement damage → Municipal - PWD (Roads)\n'
        f'5. Broken street lights, lamp posts, unlit road → Municipal - Street Lighting\n'
        f'6. Fallen trees, overgrown parks, dead plants → Municipal - Horticulture\n'
        f'7. Smoke, burning, industrial pollution → Pollution Control Board\n'
        f'8. Illegal parking, encroachment → Police - Local Law Enforcement\n'
        f'9. Traffic signal failure, road blockage → Police - Traffic\n'
        f'10. Damaged bus shelter, state bus → State Transport\n'
        f'11. Black/blurry/selfie/food → Uncategorized\n\n'
        f'Do NOT default to Water & Sewerage unless water/flooding/drain is explicit.\n'
        f'Litter/waste on road = Sanitation. Pole with wires = Power (DISCOM).\n\n'
        f'Respond with JSON: {{"group": "<group>", "department": "<exact category name>", '
        f'"confidence": <0.0-1.0>, "rationale": "<brief reason>"}}'
    )


def _build_single_pass_prompt() -> str:
    """Prompt for single-pass vision model classification with JSON output."""
    categories_block = '\n'.join(
        f'- {cat}: {CATEGORY_DEFINITIONS[cat]}' for cat in CANONICAL_CATEGORIES
    )
    return (
        f'You are a civic complaint classifier for Indian municipal authorities.\n'
        f'Look at the image and classify it into the SINGLE best matching category.\n\n'
        f'Categories:\n{categories_block}\n\n'
        f'Decision rules:\n'
        f'1. Dangling wires, power cables, transformer → Utility - Power (DISCOM)\n'
        f'2. Waterlogging, flood, drain overflow, sewer → Municipal - Water & Sewerage\n'
        f'3. Garbage, trash, waste, dump, litter → Municipal - Sanitation\n'
        f'4. Potholes, broken road, pavement damage → Municipal - PWD (Roads)\n'
        f'5. Broken street lights, lamp posts → Municipal - Street Lighting\n'
        f'6. Fallen trees, overgrown parks → Municipal - Horticulture\n'
        f'7. Smoke, burning, industrial pollution → Pollution Control Board\n'
        f'8. Illegal parking, encroachment → Police - Local Law Enforcement\n'
        f'9. Traffic signal failure, road blockage → Police - Traffic\n'
        f'10. Damaged bus shelter, state bus → State Transport\n'
        f'11. Black/blurry/selfie/food → Uncategorized\n\n'
        f'Respond with JSON: {{"department": "<exact category name>", '
        f'"confidence": <0.0-1.0>, "rationale": "<brief reason>"}}'
    )


# ══════════════════════════════════════════════════════════════════
# CLASSIFY FUNCTION — all fixes integrated
# ══════════════════════════════════════════════════════════════════

def classify(image_path: Path) -> dict:
    """
    Classify a civic complaint image.
    
    Returns dict with:
        department:         canonical category name
        confidence:         model-reported confidence (0.0-1.0)
        method:             'single_pass' | 'reasoning' | 'keyword_fallback' | 'error'
        vision_description: free-text description from vision model (accuracy mode)
        rationale:          model's reasoning for the classification
        raw_json:           raw JSON string from the model
        is_valid:           whether this is a valid civic complaint
    """
    try:
        image_bytes = load_as_jpeg_bytes(image_path)

        # ── FAST MODE: Single-pass vision classification ──────────
        if SINGLE_PASS:
            resp = ollama.generate(
                model=VISION_MODEL,
                format='json',              # FIX: enforces JSON output
                prompt=_build_single_pass_prompt(),
                images=[image_bytes],
                options={'num_ctx': 2048, 'temperature': 0.0},
            )
            raw_json = resp['response'].strip()
            method = 'single_pass'
            description = ''

            try:
                parsed = json.loads(raw_json)
                raw_label = str(parsed.get('department', 'Uncategorized'))
                confidence = float(parsed.get('confidence', 0.5))
                rationale = str(parsed.get('rationale', ''))
            except (json.JSONDecodeError, ValueError, TypeError):
                raw_label = raw_json
                confidence = 0.5
                rationale = ''

            canonical = canonicalize(raw_label)

            # Keyword fallback on rationale text if Uncategorized
            if canonical == 'Uncategorized':
                kw_text = rationale + ' ' + raw_label
                kw_result = keyword_fallback(kw_text)
                if kw_result != 'Uncategorized':
                    canonical = kw_result
                    method = 'keyword_fallback'
                    confidence = min(confidence, 0.65)

            is_valid = canonical != 'Uncategorized'
            return {
                'department': canonical,
                'confidence': confidence if is_valid else max(confidence * 0.5, 0.1),
                'method': method,
                'vision_description': description,
                'rationale': rationale,
                'raw_json': raw_json,
                'is_valid': is_valid,
            }

        # ── ACCURACY MODE: Two-step Vision→Reasoning ──────────────
        # Step 1 — Vision: Describe the image (free text)
        vision_resp = ollama.generate(
            model=VISION_MODEL,
            prompt=(
                'You are analyzing a civic complaint photo from India. '
                'Describe what you see in 2-3 factual sentences. '
                'Focus on: what is visibly damaged or problematic, '
                'the setting (road, park, building, drain, etc.), '
                'and any visible hazards or health/safety risks. '
                'Be specific and objective. Do not greet or explain yourself.'
            ),
            images=[image_bytes],
            options={'num_ctx': 2048, 'temperature': 0.2},
        )
        description = vision_resp['response'].strip()

        # Step 2 — Reasoning: Hierarchical JSON classification
        assert REASONING_MODEL is not None, 'REASONING_MODEL must be set in accuracy mode'
        reasoning_resp = ollama.generate(
            model=REASONING_MODEL,
            format='json',                  # FIX: enforces JSON output
            options={'num_ctx': 2048, 'temperature': 0.1},
            prompt=_build_reasoning_prompt(description),
        )
        raw_json = reasoning_resp['response'].strip()
        method = 'reasoning'

        # Parse structured JSON
        try:
            parsed = json.loads(raw_json)
            raw_label = str(parsed.get('department', 'Uncategorized'))
            confidence = float(parsed.get('confidence', 0.5))
            rationale = str(parsed.get('rationale', ''))
        except (json.JSONDecodeError, ValueError, TypeError):
            raw_label = raw_json
            confidence = 0.5
            rationale = ''

        canonical = canonicalize(raw_label)

        # Keyword fallback if reasoning returned Uncategorized
        if canonical == 'Uncategorized':
            kw_result = keyword_fallback(description)
            if kw_result != 'Uncategorized':
                canonical = kw_result
                method = 'keyword_fallback'
                confidence = min(confidence, 0.65)

        # Check if image is non-civic
        desc_lower = description.lower()
        is_non_civic = any(kw in desc_lower for kw in _NEGATIVE_KEYWORDS)
        is_valid = (canonical != 'Uncategorized') and not is_non_civic

        return {
            'department': canonical,
            'confidence': confidence if is_valid else max(confidence * 0.5, 0.1),
            'method': method,
            'vision_description': description,
            'rationale': rationale,
            'raw_json': raw_json,
            'is_valid': is_valid,
        }

    except Exception as e:
        return {
            'department': 'Uncategorized',
            'confidence': 0.0,
            'method': 'error',
            'vision_description': f'ERROR: {e}',
            'rationale': '',
            'raw_json': '',
            'is_valid': False,
        }

print(f'✅ Classifier ready (mode={MODE}, json_output=True, keyword_fallback=True, hierarchical=True)')

## 4 — Run re-sort (with full audit trail)

Reads images from the Kaggle input datasets and writes sorted copies to `/kaggle/working/ai_sorted_dataset/`.

> **Note:** Kaggle input datasets are read-only, so images are **copied** (not moved).

### Audit columns in the CSV
| Column | Description |
|--------|-------------|
| `method` | `reasoning`, `single_pass`, `keyword_fallback`, or `error` |
| `confidence` | Model-reported confidence 0.0–1.0 |
| `rationale` | Model's reasoning for classification |
| `raw_json` | Complete raw JSON from the model |
| `vision_description` | Free-text image description (accuracy mode) |

In [ ]:
import csv
import random
import shutil
from tqdm.notebook import tqdm

# ── Controls ──────────────────────────────────────────────────────
SAMPLE_PER_FOLDER = 5    # 0 = process all; >0 = sample N per folder for testing
RESUME = True
MAX_IMAGES_PER_RUN = 0   # 0 = process all remaining

random.seed(42)

# ── Collect images ────────────────────────────────────────────────
def collect_images(sources: list, sample: int) -> list:
    images = []
    for source in sources:
        for folder in sorted(source.iterdir()):
            if not folder.is_dir():
                continue
            found = [p for p in folder.iterdir() if p.suffix.lower() in IMAGE_EXTS]
            if not found:
                continue
            picked = random.sample(found, min(sample, len(found))) if sample else found
            images.extend(picked)
    return images

def safe_copy(src: Path, dest_dir: Path) -> Path:
    dest_dir.mkdir(parents=True, exist_ok=True)
    dest = dest_dir / src.name
    if dest.exists():
        counter = 1
        while dest.exists():
            dest = dest_dir / f'{src.stem}_{counter}{src.suffix}'
            counter += 1
    shutil.copy2(src, dest)
    return dest

# ── Resume support ────────────────────────────────────────────────
processed = set()
if RESUME and REPORT_CSV.exists():
    with open(REPORT_CSV, 'r', newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            src = row.get('source_path')
            if src:
                processed.add(src)
    print(f'Resume enabled: {len(processed)} already processed')

images = collect_images(INPUT_ROOTS, SAMPLE_PER_FOLDER)
print(f'Total images to process: {len(images)}')

OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

# ── Expanded CSV headers (full audit trail) ───────────────────────
headers = [
    'filename', 'source_path', 'source_folder', 'original_label',
    'ai_label', 'confidence', 'method', 'rationale',
    'vision_description', 'raw_json', 'is_valid', 'dest_path',
]

FOLDER_TO_LABEL = {
    'Municipal_-_PWD_Roads':          'Municipal - PWD (Roads)',
    'Municipal_-_Sanitation':         'Municipal - Sanitation',
    'Municipal_-_Horticulture':       'Municipal - Horticulture',
    'Municipal_-_Street_Lighting':    'Municipal - Street Lighting',
    'Municipal_-_Water_and_Sewerage': 'Municipal - Water & Sewerage',
    'Utility_-_Power_DISCOM':         'Utility - Power (DISCOM)',
    'State_Transport':                'State Transport',
    'Pollution_Control_Board':        'Pollution Control Board',
    'Police_-_Local_Law_Enforcement': 'Police - Local Law Enforcement',
    'Police_-_Traffic':               'Police - Traffic',
    'Uncategorized':                  'Uncategorized',
}

# ── Stats ──────────────────────────────────────────────────────────
moved = same = errors = skipped = processed_this_run = 0
method_counts = {}  # track how each classification method is used

file_mode = 'a' if RESUME and REPORT_CSV.exists() and REPORT_CSV.stat().st_size > 0 else 'w'
write_header = file_mode == 'w'

# ── Main classification loop ──────────────────────────────────────
with open(REPORT_CSV, file_mode, newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    if write_header:
        writer.writeheader()

    for img_path in tqdm(images, desc='Re-sorting', unit='img'):
        if RESUME and str(img_path) in processed:
            skipped += 1
            continue

        folder_name = img_path.parent.name
        original_label = FOLDER_TO_LABEL.get(folder_name, folder_name.replace('_', ' '))

        try:
            result = classify(img_path)
            ai_label    = result['department']
            confidence  = result['confidence']
            method      = result['method']
            rationale   = result.get('rationale', '')
            vision_desc = result.get('vision_description', '')
            raw_json    = result.get('raw_json', '')
            is_valid    = result.get('is_valid', True)

            if confidence <= 0.1 or ai_label in ('Unknown', ''):
                ai_label = 'Uncategorized'

            dest_dir = OUTPUT_ROOT / safe_dirname(ai_label)
            dest_path = safe_copy(img_path, dest_dir)

            if ai_label != original_label:
                moved += 1
            else:
                same += 1

            method_counts[method] = method_counts.get(method, 0) + 1

        except Exception as e:
            errors += 1
            ai_label = 'Uncategorized'
            confidence = 0.0
            method = 'error'
            rationale = ''
            vision_desc = f'ERROR: {e}'
            raw_json = ''
            is_valid = False
            dest_path = ''

        writer.writerow({
            'filename':           img_path.name,
            'source_path':        str(img_path),
            'source_folder':      folder_name,
            'original_label':     original_label,
            'ai_label':           ai_label,
            'confidence':         f'{confidence:.3f}',
            'method':             method,
            'rationale':          rationale[:200],  # truncate for CSV readability
            'vision_description': vision_desc[:300],
            'raw_json':           raw_json[:500],
            'is_valid':           is_valid,
            'dest_path':          str(dest_path),
        })
        processed_this_run += 1

        if MAX_IMAGES_PER_RUN and processed_this_run >= MAX_IMAGES_PER_RUN:
            break

print(f'\n✅ Done!')
print(f'   Processed this run : {processed_this_run}')
print(f'   Skipped (resume)   : {skipped}')
if processed_this_run:
    print(f'   Re-labelled        : {moved}  ({moved/processed_this_run*100:.1f}%)')
    print(f'   Confirmed          : {same}  ({same/processed_this_run*100:.1f}%)')
print(f'   Errors             : {errors}')
print(f'   Report CSV         : {REPORT_CSV}')
print(f'\n   Methods used:')
for m, c in sorted(method_counts.items(), key=lambda x: -x[1]):
    print(f'     {m:<25} {c:>5}  ({c/processed_this_run*100:.1f}%)') if processed_this_run else None

## 5 — Output summary & explainability analysis

In [ ]:
import pandas as pd

df = pd.read_csv(REPORT_CSV)

print('=' * 60)
print('  CLASSIFICATION REPORT')
print('=' * 60)

print('\n📊 Per-category output counts:')
print(df['ai_label'].value_counts().to_string())

print('\n🔧 Classification method breakdown:')
method_stats = df['method'].value_counts()
for m, c in method_stats.items():
    print(f'  {m:<25} {c:>5}  ({c/len(df)*100:.1f}%)')

print(f'\n📈 Confidence statistics:')
df['confidence'] = pd.to_numeric(df['confidence'], errors='coerce')
print(f'  Mean confidence  : {df["confidence"].mean():.3f}')
print(f'  Median confidence: {df["confidence"].median():.3f}')
print(f'  Min confidence   : {df["confidence"].min():.3f}')
print(f'  Max confidence   : {df["confidence"].max():.3f}')

# Confidence distribution by method
print(f'\n  Confidence by method:')
for method in df['method'].unique():
    subset = df[df['method'] == method]['confidence']
    print(f'    {method:<25} mean={subset.mean():.3f}  median={subset.median():.3f}')

print('\n🔀 Re-labelled from → to (top 20):')
relabelled = df[df['original_label'] != df['ai_label']]
if not relabelled.empty:
    print(relabelled.groupby(['original_label', 'ai_label']).size()
          .sort_values(ascending=False).head(20).to_string())
else:
    print('  (none re-labelled)')

# Uncategorized analysis
uncat = df[df['ai_label'] == 'Uncategorized']
print(f'\n⚠️  Uncategorized: {len(uncat)} / {len(df)} ({len(uncat)/len(df)*100:.1f}%)')
if not uncat.empty:
    print(f'  Original labels of uncategorized images:')
    print(uncat['original_label'].value_counts().head(10).to_string())

# Keyword fallback analysis
kw_fallback = df[df['method'] == 'keyword_fallback']
if not kw_fallback.empty:
    print(f'\n🔑 Keyword fallback rescued {len(kw_fallback)} images:')
    print(kw_fallback['ai_label'].value_counts().to_string())

print('\n📁 Output folder sizes:')
for cat_dir in sorted(OUTPUT_ROOT.iterdir()):
    if cat_dir.is_dir():
        n = sum(1 for _ in cat_dir.iterdir())
        print(f'  {cat_dir.name:<45} {n:>5}')

print(f'\n📄 Full report: {REPORT_CSV}')